# Modelgrid and intersection

The first part of this notebook is focused on the `flopy.discretization.Grid` (modelgrid) object(s). The modelgrid object(s) are a relatively new addition to FloPy's capabilities and are the backbone of plotting, exporting, and GIS data processing within FloPy. These objects are automatically created when a model is loaded. Alternatively they can also be created as a stand alone object. 

There are three types of modelgrids:

* `StructuredGrid`: the StructuredGrid object is created for rectilinear grids. i.e. models that use a DIS file for discretization
* `VertexGrid`: the VertexGrid object is for discretizations that are defined by vertices (e.g., DISV packages)
* `UnstructuredGrid`: the UnstructuredGrid object is for unstructured discretizations (e.g., DISU and MODFLOW-USG)

These objects all have a common interface defined by the base `Grid` class. What this means for the user is if they want to get the cell center coordinates from a modelgrid, the function call is identical on all three grids.

The best way to learn about these classes is by checking them out. 

In [ ]:
import pathlib as pl
import flopy
from flopy.utils.gridgen import Gridgen
from flopy.utils import GridIntersect, Raster

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

# pd.options.mode.chained_assignment = None
data_path = pl.Path("data/modelgrid_intersection")

## Grids

## StructuredGrid

Let's start with the most common type of MODFLOW modelgrid, the `StructuredGrid`

#### First we'll build one from scratch 

In [ ]:
nrow = 20
ncol = 15
nlay = 1

grad = np.expand_dims(np.linspace(50, 100, nrow), axis=1)[::-1]
top = np.ones((nrow, ncol)) * grad
botm = np.zeros((nlay, nrow, ncol))
botm[0, :, :] = top - 50 
delr = np.full((ncol,), 250)
delc = np.full((nrow,), 150)
ibound = np.ones((nlay, nrow, ncol))
ibound[0, 4:10, 4:8] = 0

modelgrid = flopy.discretization.StructuredGrid(
    delc,
    delr,
    top,
    botm,
    ibound,
)
print(modelgrid)
print(type(modelgrid))

Let's take a look at our modelgrid visually

In [ ]:
modelgrid.plot();

This is great, but what if we want to orient it in space and add a coordinate reference system so we can do further processing? 

The `set_coord_info()` method allows us to update the coordinate offsets, rotation of the grid, and even add projection information.

In [ ]:
xoff = 2345678
yoff = 1234567
angrot = -15
epsg = 32610  # utm zone 10N

modelgrid.set_coord_info(xoff=xoff, yoff=yoff, angrot=angrot, crs=epsg)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax = modelgrid.plot(ax=ax)
plt.title(str(modelgrid));

### Getting cell centers and cell vertices

* Cell centers can be returned to the user through the `.xcellcenters`, `.ycellcenter`, and `.zcellcenter` properties.
* Cell vertices can be returned to the user through the `.xvertices` and `.yvertices` properties

In [ ]:
xc, yc = modelgrid.xcellcenters, modelgrid.ycellcenters
xv, yv = modelgrid.xvertices, modelgrid.yvertices

fig, ax = plt.subplots(figsize=(10, 10))
modelgrid.plot()
ax.scatter(xc.ravel(), yc.ravel(), c="b", label="cell centers")
ax.scatter(xv.ravel(), yv.ravel(), c="r", label="cell vertices")
plt.legend(loc=0);

### Getting the ibound/idomain array

The `.idomain` method returns the ibound/idomain array, which defines active (>1), inactive (0), and vertical pass through cells (<0) in the model.

In [ ]:
ibound = modelgrid.idomain

fig, ax = plt.subplots(figsize=(8, 8))
pmv = flopy.plot.PlotMapView(modelgrid=modelgrid, ax=ax)
pmv.plot_grid()
pmv.plot_array(ibound, masked_values=[0,]);

### Getting the model top, bottom, delc, and delr

The model top, bottom, delc, and delr can be accessed from the modelgrid using `.top`, `.bottom`, `.delc`, and `.delr`.

In [ ]:
top = modelgrid.top
botm = modelgrid.botm
delc = modelgrid.delc
delr = modelgrid.delr
top, botm, delc, delr

## Class exercise 1:

Assume that the modelgrid is currently in meters and we want our grid to be in feet. Create a new modelgrid object from the existing one where the discretization is in feet.

make a plot of the new modelgrid after it has been created

### Finding a cell's nearest neighbors

Imagine that we want to find the nearest neighbors of a cell for some reason. The modelgrid has a built in method to 1) internally construct a table of nearest neighbors and 2) return the nearest neighbors for a given cell to the user.

In [ ]:
cell = (0, 0, 5)

neighbors = modelgrid.neighbors(*cell)
neighbors

### Finding grid cells by point location

If we want to find a cell number based on x, y coordinate location the model grid has a built in `.intersect()` method that takes `x`, `y`, and an optional `z` coordinate and returns the cell number 

In [ ]:
x, y, z = 2348000, 1235000, 25

rowcol = modelgrid.intersect(x, y) # without z
layrowcol = modelgrid.intersect(x, y, z) # with z location

f"{rowcol=}, {layrowcol=}"

## Vertex model grid

The vertex model grid is produced for MODFLOW model discretizations that are defined by vertices (`DISV`). We can build a quadtree vertex grid from an existing structured modelgrid using the gridgen utility and the `Gridgen` class.

First let's unrotate the grid coordinates of our structured grid to make this easy

In [ ]:
modelgrid.set_coord_info(angrot=0)
modelgrid.plot();

Lets load a couple of shapefiles to help with creating the quadtree

In [ ]:
active_shp = data_path / "active_area.shp"
refine_shp = data_path / "refined_area.shp"

active = gpd.read_file(active_shp)
refined = gpd.read_file(refine_shp)

Now let's plot these areas on the grid to visualize them

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

pmv = flopy.plot.PlotMapView(modelgrid=modelgrid)
pmv.plot_grid()
# active.geometry.to_list()
pmv.plot_shapes(active.geometry.to_list(),  alpha=0.5, cmap='plasma')
pmv.plot_shapes(refined.geometry.to_list(), alpha=0.5, cmap='magma');

### Generate the quadtree grid using `Gridgen`

In [ ]:
g = Gridgen(modelgrid, model_ws=data_path)
g.add_refinement_features(refined.geometry.to_list(), "polygon", 1, [0,])
g.build(verbose=False)

#### Create a vertex model grid from the results

In [ ]:
gridprops = g.get_gridprops_vertexgrid()
vertexgrid = flopy.discretization.VertexGrid(**gridprops)
vertexgrid.plot();

### Getting the modelgrid's shape


In [ ]:
shape = vertexgrid.shape
shape

## Class Exercise 2: 

Find and plot all of the nearest neighbors at node 79 on the vertex modelgrid.

**Hint**: Use `flopy.plot.PlotMapView` for your plotting, and plot a boolean (1, 0) array with the cells that are not neighbors masked out.


## UnstructuredGrid

The `UnstructuredGrid` class is used with MODFLOW-USG and MODFLOW-6 DISU type discretizations. We can also create an `UnstructuredGrid` object from the `Gridgen` results

In [ ]:
gridprops = g.get_gridprops_unstructuredgrid()

ugrid = flopy.discretization.UnstructuredGrid(**gridprops)
ugrid, type(ugrid)

In [ ]:
ugrid.plot();

In [ ]:
neighbors = ugrid.neighbors(79)
arr = np.zeros(ugrid.shape, dtype=int)
arr[neighbors] = 1

fig, ax = plt.subplots(figsize=(8, 8))

pmv = flopy.plot.PlotMapView(ax=ax, modelgrid=ugrid)
pmv.plot_grid()
pmv.plot_array(arr, masked_values=[0,]);

## Intersecting geospatial data

In this part of the lesson we will intersect geospatial data with a modelgrid and begin building a simple adaptation of the Sagehen Creek model, near Truckee, CA. While building this model, we'll work with shapefile and raster data. As we build the model FloPy's intersection capabilities will be presented.

Let's start by getting the basin boundary. For the Sagehen Creek model, we're interested in the contributing area for [USGS gage 10343500](https://waterdata.usgs.gov/ca/nwis/uv/?site_no=10343500&PARAmeter_cd=00065,00060) 

In [ ]:
station_id = "10343500"
basin = gpd.read_file(data_path / "sagehen_basin.shp")
basin = basin.set_index("identifier")
basin

First thing that we notice is that the basin polygon is projected in decimal degrees, which is not good for modeling. An equal area projection would be better suited for this application

In [ ]:
epsg = 26911  # NAD83 utm zone 11 N, epsg: 26911
basin = basin.to_crs(epsg=epsg)
basin

In [ ]:
basin.plot()

We can now get our basin's bounding box using `.bounds`, buffer it, and then begin creating a modelgrid based on the basin boundary.

In [ ]:
cellsize = 90  # 90 m grid
bounds = basin.bounds
print(bounds)
xmin = bounds.loc[station_id, "minx"] - (cellsize * 1)
ymin = bounds.loc[station_id, "miny"] - (cellsize * 1)
xmax = bounds.loc[station_id, "maxx"] + (cellsize * 1)
ymax = bounds.loc[station_id, "maxy"] + (cellsize * 1)

Calculate the number of rows and columns the model will have and then create a delc and delr array.

In [ ]:
dx = xmax - xmin
dy = ymax - ymin

nlay = 1
nrow = np.ceil(dy / cellsize).astype(int)
ncol = np.ceil(dx / cellsize).astype(int)
delr = np.full((ncol,), cellsize)
delc = np.full((nrow,), cellsize)

# let's create a fake top and bottom for now
top = np.full((nrow, ncol), 100)
botm = np.full((nlay, nrow, ncol), 0)

Now we can create a minimal modelgrid from the information we have

In [ ]:
modelgrid = flopy.discretization.StructuredGrid(
    delc,
    delr,
    top=top,
    botm=botm,
    nlay=nlay,
    xoff=xmin,
    yoff=ymin,
    crs=epsg
)
print(modelgrid)
print(modelgrid.is_complete)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

ax = basin.plot(ax=ax)
modelgrid.plot(ax=ax, lw=0.5);

Fantastic! The modelgrid has been created and it completely overlays the contributing area.

### Intersecting a polygon with the modelgrid

FloPy has a utility named `GridIntersect` that allows the user to intersect points, polygons, and polylines with modelgrid objects. 

In this example, the basin boundary will be intersected with the modelgrid to create the active and inactive extents of the model. The `.intersect()` method returns a numpy recarry with cellids, vertices, cell areas, and shapely polygon objects.

In [ ]:
gx = GridIntersect(modelgrid)

result = gx.intersect(basin.loc[station_id, "geometry"])
result.cellids

create the ibound/idomain array (active and inactive model cells)

In [ ]:
ibound = np.zeros((nlay, nrow, ncol))
i, j = zip(*result.cellids)
ibound[:, i, j] = 1

fig, ax = plt.subplots(figsize=(8, 8))
pmv = flopy.plot.PlotMapView(modelgrid=modelgrid, ax=ax)
pmv.plot_grid(lw=0.5)
pmv.plot_array(ibound, masked_values=[0], cmap="viridis")

### Intersecting raster data with a modelgrid

FloPy's `Raster` class allows us to intersect and resample raster data to the model grid. This class is useful for resampling a number of gridded products such as: Digital Elevation Model data, land cover data, and gridded climate data.

The first example of using the `Raster` class will be to resample Digital Elevation Model (DEM) data to define the land surface of the Sagehen model

In [ ]:
dem_file = data_path / "dem_30m.img"

raster = Raster.load(dem_file)

#### Now that the raster is loaded we can resample it

The `resample_to_grid()` method performs geostatistics and returns an array in the shape of the modelgrid. Options for resampling include `"nearest"`, `"linear"` (bilinear), `"cubic"` (bicubic), `"max"`, `"min"`, `"mean"`, `"median"`, and `"mode"` (most common value).

In [ ]:
# let's get the nearest neighbor elevations
dem_data = raster.resample_to_grid(
    modelgrid,
    band=raster.bands[0],
    method="nearest"
)
dem_data

In [ ]:
# Now let's plot it up and see what we have
fig, ax = plt.subplots(figsize=(10, 8)) 
pmv = flopy.plot.PlotMapView(ax=ax, modelgrid=modelgrid)

pmv.plot_grid(lw=0.5, zorder=3)
pc = pmv.plot_array(dem_data, alpha=0.75, zorder=1, cmap="viridis")
pmv.plot_array(ibound, masked_values=[1], cmap="magma", zorder=2)
plt.colorbar(pc, shrink=0.7);

### Start creating model files from the data we've gathered so far.

At this point we have almost everything we need to begin creating a model discretization package. The only thing left to calculate is the model bottom. Let's assume bedrock 150 m below land surface throughout the basin and begin creating our model. 


We'll first start by creating our simulation object and adding a solver and the time discretization. This model will be a year long with 12 stress periods and a daily time step

In [ ]:
sim = flopy.mf6.MFSimulation("sagehen", sim_ws=data_path)
ims = flopy.mf6.ModflowIms(sim, complexity="COMPLEX")
nper = 12
perlen = (31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31)
period_data = [(i, i, 1.0) for i in perlen]
tdis = flopy.mf6.ModflowTdis(
    sim,
    nper=12,
    perioddata=period_data,
    time_units="days"
)

Now to create the Groundwater flow model object and add the dis file to it

In [ ]:
gwf = flopy.mf6.ModflowGwf(
    sim,
    modelname='sagehen',
    save_flows=True,
)

# add the dis package
botm = np.expand_dims(dem_data, axis=0) - 150
dis = flopy.mf6.ModflowGwfdis(
    gwf,
    nrow=modelgrid.nrow,
    ncol=modelgrid.ncol,
    delr=modelgrid.delr,
    delc=modelgrid.delc,
    top=dem_data,
    botm=botm,
    idomain=ibound
)

gwf

We can also add an initial conditions file to the model. Let's assume water level begins at 5 meters below land surface

In [ ]:
ic = flopy.mf6.ModflowGwfic(gwf, strt=dem_data - 5)
gwf

And we can create our NPF and STO packages using values from literature

In [ ]:
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    save_flows=True,
    save_specific_discharge=True,
    k=0.022  # value from Larsen et al, 2022
)

sto = flopy.mf6.ModflowGwfsto(
    gwf,
    save_flows=True,
    iconvert=1,
    ss=1e-07,  # values from Larsen et al, 2022
    sy=0.2
)

We can also add a OC package now

In [ ]:
# build output control package
budget_file = "sagehen.cbc"
head_file = "sagehen.hds"
saverecord = {i: [("HEAD", "ALL"), ("BUDGET", "ALL")] for i in range(gwf.nper)}
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    budget_filerecord=budget_file,
    head_filerecord=head_file,
    saverecord=saverecord,
)

## Class Exercise: Loading and resampling rasters

For this exercise we are loading and resampling monthly Daymet climate data for Sagehen basin.

Load the potential evapotranspiration raster `pet_raster` using flopy's `Raster` class and create a dictionary of resampled PET that stores all 12 months of values.

**Hints:**

   - Rasters contain multiple datasets called bands, the `Raster` class has a `.bands` attribute that returns a list of all of the bands within the raster. Lists can be looped over!
   - Because the data is coarse compared to our model discretization, the nearest neighbor method should be appropriate for resampling `method="nearest"`
   - Finally no data values should be converted to 0 value. The `Raster` class has a `.nodatavals` attribute that returns a list of no data values. There is only one in this list; use numpy's boolean indexing or `np.where()` to replace all of the no data entries with 0.

In [ ]:
pet_raster = data_path / "pet.tif"
prcp_raster = data_path / "prcp.tif"

Let's load the saved version of this data, it should be identical to the data that was produced in the exercise.

In [ ]:
arr = np.genfromtxt(data_path / "pet.txt")
arr.shape = (12, modelgrid.nrow, modelgrid.ncol)
pet_monthly = {i: a for i, a in enumerate(arr)}
pet_monthly

And resample the precipitation raster we saved earlier

In [ ]:
rstr = flopy.utils.Raster.load(prcp_raster)
prcp_monthly = {}
for band in rstr.bands:
    t = rstr.resample_to_grid(
        modelgrid,
        band,
        method="nearest"
    )
    t[t == rstr.nodatavals[0]] = 0
    prcp_monthly[band - 1] = t
    
prcp_monthly

Potetial evapotranspiration and Precipitation data from daymet are in mm/day, however our discretization is meters/day. 

In [ ]:
prcp_monthly = {k: v * 0.001 for k, v in prcp_monthly.items()}
pet_monthly = {k: v * 0.001 for k, v in pet_monthly.items()}

Finally we can load a saved soil hydraulic conductivity array, and we have the information we need to create a Unsaturated Zone Flow (UZF) package for this model

In [ ]:
ksat_raster = data_path / "ksat.img"
rstr = flopy.utils.Raster.load(ksat_raster)
vks = rstr.resample_to_grid(
    modelgrid,
    rstr.bands[0],
    method="nearest"
)

vks[vks == rstr.nodatavals[0]] = np.nan
vks[np.isnan(vks)] = np.nanmean(vks)
vks *= 0.0864  # um per sec to m per day
ims = plt.imshow(vks)
plt.colorbar(shrink=0.6); 

#### Creating a UZF package 

Now that we have some climate and soils data we can create a UZF package from it and add it to the model!

Create the package data and the stress period data info for flopy

For the package data we are making some basic assumptions. `landflag=1` and `ivertcon=0` meaning that the cell is at the surface and is not connected to an underlying uzf cell. The `surfdep=1.0` which means that we have a surface depression (variable land surface) of 1 meter for all cells. `thtr=0.1`, `thts=0.38` meaning the soil is fairly porous and transmits water readily. `thti=0.1` which means initial conditions are residual water content. And finally a brooks-corey epsilon `eps=3.5`. 

For the stress period data, some additional assumptions are made. `extdp=0.5` which means that the evapotranspiration extinction depth is 0.5 m below the surface. `extwc=0.2` which means that the extinction water content (low end of field capacity) is 0.2 (unitless or m^3/m^3). All other values are placeholders and are not used with the current simulation options.  

For more information about the UZF input options please see [UZF: online guide to MF6](https://modflow6.readthedocs.io/en/latest/_mf6io/gwf-uzf.html)

In [ ]:
modelgrid = gwf.modelgrid
modelgrid.set_coord_info(xoff=xmin, yoff=ymin)
package_data = []
cnt = 0
for i in range(modelgrid.nrow):
    for j in range(modelgrid.ncol):
        if modelgrid.idomain[0, i, j] == 0:
            continue
        # ifno, cellid, landflag, ivertcon, surfdep, vks, thtr, thts, thti, eps
        rec = (cnt, (0, i, j), 1, 0, 1.0, vks[i, j], 0.1, 0.38, 0.1, 3.5)
        package_data.append(rec)
        cnt += 1

period_data = {}
for per in range(gwf.nper):
    spd = []
    cnt = 0
    for i in range(modelgrid.nrow * modelgrid.ncol):
        if modelgrid.idomain.ravel()[i] == 0:
            continue
        # ifno, finf, pet, extdp, extwc, [ha, hroot, rootact] in record, but not simulated
        rec = (cnt, prcp_monthly[per].ravel()[i], pet_monthly[per].ravel()[i], 0.5, 0.2, -1.1, -75, 1.0)
        spd.append(rec)
        cnt += 1
        
    period_data[per] = spd

In [ ]:
uzf = flopy.mf6.ModflowGwfuzf(
    gwf,
    simulate_et=True,
    ntrailwaves=15,
    nwavesets=100,
    packagedata=package_data,
    perioddata=period_data,
    unsat_etwc=True,
    linear_gwet=True,
    simulate_gwseep=True
)

### Now to create a simple river package from NHD flowlines 

We can use the intersection routines that have been presented thus far to create a river package for the sagehen model. 

The first step is load the NHD flowlines for the basin

In [ ]:
flow_lines = data_path / "sagehen_nhd.shp"
flw_all = gpd.read_file(flow_lines)
flw_all.plot();

Excellent! Now we have all of the flow lines for the Sagehen watershed. But what if we want to identify the main stem? 

In [ ]:
flow_lines = data_path / "sagehen_main_nhd.shp"
flw_main = gpd.read_file(flow_lines)
flw_main.plot()
flw_main

Great! Now we can start processing data to create a river package. We can first identify our main stem cells.

In [ ]:
main_stem = []
gx = GridIntersect(modelgrid)
for iloc, row in flw_main.iterrows():
    results = gx.intersect(row.geometry)
    main_stem += list(results.cellids)
main_stem[0:5]

## Class Exercise:

Use GridIntersect to create a list of tributary cells from the `flw_all` geodataframe. 

**Hint**
Use the `main_stem` list to filter out cells that are associated with the main stem of the river and only store the cells that are associated with tributaries to the main stem.

The cellids for tributaries have been stored previously, so we'll load those up and continue on.


In [ ]:
tributaries = []
with open(data_path / "trib_cells.txt") as foo:
    for line in foo:
        t = line.strip().split(",")
        tributaries.append((int(t[0]), int(t[1])))

#### We can also get daily discharge data to help calculate average monthly stages for the RIV package

We'll use the `dataretrieval` package to get stage information for our model

In [ ]:
import dataretrieval.nwis as nwis

gage = nwis.get_record(sites=station_id, service='iv', start="2023-01-01", end="2023-12-31", parameterCd="00065")
gage.to_csv(data_path / "sagehen_gage_data.csv")

In [ ]:
gage = pd.read_csv(data_path / "sagehen_gage_data.csv")
gage["datetime"] = pd.to_datetime(gage["datetime"])
gage = gage.set_index("datetime")

gage.rename(columns={"00065": "stage"}, inplace=True)
gage.stage *= 0.3284
ax = gage.stage.plot()
ax.set_ylabel("stage, in feet")
gage.head()

Now we can aggregate to a mean monthly stage from 15 minute instantaneous values

In [ ]:
gage["month"] = gage.index.month
gage_mmo = gage.groupby(by=["month"], as_index=False)["stage"].mean()
ax = gage_mmo.stage.plot()
ax.set_ylabel("mean stage, in feet");

### Now to create the RIV package


In [ ]:
cond = 10
rbadj = 1
stage_adj = 0.33

perioddata = {}
for kper, row in gage_mmo.iterrows():
    spd = []
    for (i, j) in main_stem:
        ctop = modelgrid.top[i, j]
        rbed = ctop - rbadj
        rec = ((0, i, j), row.stage + rbed, cond, rbed)
        spd.append(rec)
        
    for (i, j) in tributaries:
        ctop = modelgrid.top[i, j]
        rbed = ctop - (rbadj / 2)
        rec = ((0, i, j), row.stage + rbed, cond, rbed)
        spd.append(rec)
    
    perioddata[kper] = spd

perioddata[0][0:5]

In [ ]:
riv = flopy.mf6.ModflowGwfriv(
    gwf,
    save_flows=True,
    maxbound=len(perioddata[0]),
    stress_period_data=perioddata
)


# Testing the model

In [ ]:
sim.write_simulation()
sim.run_simulation();

In [ ]:
hds = gwf.output.head()
heads = hds.get_alldata()[-1]

fig, ax = plt.subplots(figsize=(10, 8))
pmv = flopy.plot.PlotMapView(modelgrid=modelgrid, ax=ax)
pc = pmv.plot_array(heads, cmap='viridis', masked_values=[1e30, -1e30])
pmv.plot_grid(lw=0.5)
pmv.plot_inactive()
plt.colorbar(pc, shrink=0.7)
plt.show();

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.set_aspect("equal")
xc = flopy.plot.PlotCrossSection(modelgrid=modelgrid, ax=ax, line={"row": 36}, geographic_coords=True)
pc = xc.plot_array(heads, cmap='plasma', head=heads, masked_values=[1e+30])
xc.plot_grid(lw=0.5)
xc.plot_inactive()
plt.colorbar(pc, shrink=0.50)
plt.tight_layout();